In [1]:
import numpy as np
import z3
from analysis import *

In [2]:
def decorated_scenario(summary_dict, scenario, eps, X=None, Y=None, label="UNNAMED"):
    vacuous_assertion = Polyhedron(np.asarray([]), np.asarray([]), label="true")
    scen = DecoratedPAssignNode(
        X if X is not None else vacuous_assertion,
        Y if Y is not None else vacuous_assertion,
        eps,
        summary_dict[f'A_{scenario}'],
        summary_dict[f'b_{scenario}'],
        label=label
    )
    return scen

In [3]:
summary_state_file = '../f1tenth/base_05noise_summarized_multi_init_SUMMARIES_STATES.txt'
summary_np_file = '../f1tenth/base_05noise_summarized_multi_init_SUMMARIES.npz'

f1tenth_summaries_path= '../f1tenth/base_05noise_summarized_multi_init_SUMMARIES.npz'

with open(f1tenth_summaries_path, 'rb') as f:
    f1tenth_summaries = dict(np.load(f))
def display_f1_state(state_i):
    return str(list(zip(f1tenth_summaries['states_header'], f1tenth_summaries['states_list'][state_i])))

def left_scenario(eps):
    return decorated_scenario(f1tenth_summaries, 0, eps)
def right_scenario(eps):
    return decorated_scenario(f1tenth_summaries, 1, eps)
def straight_scenario(eps):
    return decorated_scenario(f1tenth_summaries, 2, eps)

In [4]:
def choice(eps):
    scens = [ left_scenario(eps), right_scenario(eps/2), straight_scenario(eps/2)]
    vacuous_assertion = Polyhedron(np.asarray([]), np.asarray([]), label="true")
    scen = DecoratedChoiceNode(
        vacuous_assertion,
        vacuous_assertion,
        eps,
        scens,
        label='nolabel'
    )
    return scen


In [5]:
# This will check whether the guessed precondition according to the procedure in
# sectin 3.2 is invariant (aka recurrent).
recurrence_check = False
eps = 0.00
recurrent_eps = []
while eps < 1.01:
    eps = eps + 0.01
    choice_with_eps = choice(eps)
    choice_with_eps.set_non_atomic_epsilons_and_weakest_assertions(choice_with_eps.Y)
    choice_with_eps.Y = choice_with_eps.X
    choice_with_eps.set_weakest_assertions(choice_with_eps.Y)
    recurrence_check = choice_with_eps.check()
    if recurrence_check:
        recurrent_eps.append(eps)

print("The values of epsilon for which the weakest pre is invariant are:")
for e in recurrent_eps:
    print(e)

Debug: ChoiceNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
Debug: ChoiceNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
debug: child 0 of nolabel checked as True
debug: child 1 of nolabel checked as True
debug: child 2 of nolabel checked as True
i is 0
c.X.contains_other(self.X, tolerance=tolerance) evals to 
True
i is 1
c.X.contains_other(self.X, tolerance=tolerance) evals to 
True
i is 2
c.X.contains_other(self.X, tolerance=tolerance) evals to 
True
Debug: ChoiceNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
Debug: ChoiceNode called set_weakest assertions
Debug: PAssignNode called set_w

In [6]:
# For TaxiNet. This checks that the invariant the ensures a local erro probability of 0.305 is invariant works
taxinet_summaries_path = '../taxinet/bright_dark/taxinet_5bins_joint_SUMMARIES.npz'
with open(taxinet_summaries_path, 'rb') as f:
    taxinet_summaries = dict(np.load(f))
day_A = taxinet_summaries['A_0']
day_b = taxinet_summaries['b_0']
day_C = taxinet_summaries['C_0']
night_A = taxinet_summaries['A_1']
night_b = taxinet_summaries['b_1']
night_C = taxinet_summaries['C_1']

true_assertion = Polyhedron(np.asarray([]), np.asarray([]))
max_eps = 0.305
night_max_eps = DecoratedPAssignNode(true_assertion, true_assertion, max_eps, night_A, night_b)
night_max_eps_alone = DecoratedPAssignNode(true_assertion, true_assertion, max_eps, night_A, night_b)
night_max_eps_alone.set_non_atomic_epsilons_and_weakest_assertions(true_assertion)
day_max_eps = DecoratedPAssignNode(true_assertion, true_assertion, max_eps, day_A, day_b)
# # Now let's find the weakest pres
for pre, id in zip([day_max_eps.X, night_max_eps.X], ['day', 'night']):
    print(id)
    print(pre.coefficients_matrix)
    print(pre.offsets)
    print(f"Is {id} empty: {pre.is_empty()}")
choice_max_eps = DecoratedChoiceNode(true_assertion, true_assertion, max_eps, children=[day_max_eps, night_max_eps])

choice_max_eps.set_non_atomic_epsilons_and_weakest_assertions(true_assertion)



for pre, id in zip([choice_max_eps.X, choice_max_eps.children[0].X, choice_max_eps.children[1].X, night_max_eps_alone.X], ['parent', 'c0', 'c1', 'night_alone']):
    print(id)
    print(pre.coefficients_matrix)
    print(pre.offsets)
    print(f"Is {id} empty: {pre.is_empty()}")
print("is the check ok")
print(choice_max_eps.check())



Debug: PAssignNode called set_weakest assertions
day
[]
[]
Is day empty: False
night
[]
[]
Is night empty: False
Debug: ChoiceNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
Debug: PAssignNode called set_weakest assertions
parent
[[0.004 0.004 0.007 0.008 0.007 0.076 0.004 0.081 0.005 0.076 0.075 0.218
  0.081 0.21  0.08 ]
 [0.301 0.288 0.339 0.322 0.31  0.53  0.289 0.283 0.329 0.52  0.512 0.829
  0.265 0.33  0.4  ]]
[0.305 0.305]
Is parent empty: False
c0
[[0.004 0.004 0.007 0.008 0.007 0.076 0.004 0.081 0.005 0.076 0.075 0.218
  0.081 0.21  0.08 ]]
[0.305]
Is c0 empty: False
c1
[[0.301 0.288 0.339 0.322 0.31  0.53  0.289 0.283 0.329 0.52  0.512 0.829
  0.265 0.33  0.4  ]]
[0.305]
Is c1 empty: False
night_alone
[[0.301 0.288 0.339 0.322 0.31  0.53  0.289 0.283 0.329 0.52  0.512 0.829
  0.265 0.33  0.4  ]]
[0.305]
Is night_alone empty: False
is the check ok
debug: child 0 of None checked as True
debug: child 1 of None checked as True
i is 0
c.X.conta